In [8]:
import pandas as pd
import glob
import pyarrow as pa
import pyarrow.parquet as pq

files = glob.glob("../processed_data/processed_v0/*.parquet")
dfs = []

for f in files:
    df = pd.read_parquet(f)

    ts = pd.to_datetime(df["timestamp"])

    # ✅ If timestamp is tz-naive, localize to IST (+05:30)
    if ts.dt.tz is None:
        ts = ts.dt.tz_localize("Asia/Kolkata")

    # ✅ Convert everything to UTC and then drop timezone (naive UTC)
    df["timestamp"] = ts.dt.tz_convert("UTC").dt.tz_localize(None)
    df["timestamp"] = (ts.astype('int64') // 1000) # convert ns → µs
    df["timestamp"] = df['timestamp'].astype('datetime64[us]')
    dfs.append(df)

merged = pd.concat(dfs, ignore_index=True)
merged = merged.dropna(subset=["rolling_avg_10", "volume_sum_10"]).reset_index(drop=True)

merged.to_parquet("../processed_data/processed_v0/stock_features_all.parquet", index=False)
print("DONE")

DONE


In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17638842 entries, 0 to 17638841
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   timestamp       datetime64[us]
 1   open            float64       
 2   high            float64       
 3   low             float64       
 4   close           float64       
 5   volume          float64       
 6   rolling_avg_10  float64       
 7   volume_sum_10   float64       
 8   future_close    float64       
 9   target          int64         
 10  stock_symbol    object        
dtypes: datetime64[us](1), float64(8), int64(1), object(1)
memory usage: 1.4+ GB
